<a href="https://colab.research.google.com/github/GUNREDDY14/language/blob/master/Bhashathon_sathwika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch sentencepiece -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

transformers: For using T5-small.
datasets: To handle your dataset efficiently.
torch: Needed for training the model.
sentencepiece: Required for T5’s tokenizer.

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


T5Tokenizer: Tokenizes input/output text for T5.
T5ForConditionalGeneration: The actual T5-small model.
Trainer: Simplifies training using Hugging Face’s Trainer API.
Dataset: Converts Pandas data into a format Trainer can use.

In [7]:
# Install openpyxl if not installed (needed for reading Excel files)
!pip install openpyxl

import pandas as pd

# Load dataset from Excel file
df = pd.read_excel("eng_data.xlsx", engine="openpyxl")  # Ensure file name is correct

# Define input-output pairs for T5
df["input_text"] = "fix ASR: " + df["asr_output"]
df["target_text"] = df["corrected_text"]

# Drop unnecessary columns
df = df[["input_text", "target_text"]]

# Display first few rows
df.head()


,input_text,target_text
0,fix ASR: professor floyd pointed out that mass...,Professor Floyd pointed out that mass lesions...
1,fix ASR: the norsemen considered the rainbow a...,The Norsemen considered the rainbow as a brid...
2,fix ASR: she can scoop these things into three...,She can scoop these things into three red bag...
3,fix ASR: ilay is one of five whisky distilling...,Islay is one of five whisky distilling locali...
4,fix ASR: messangers from molly and molly,Messages from Molly and Mollie


In [8]:
# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)


In [10]:
from transformers import T5Tokenizer

# Load the tokenizer for T5-small
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define the function to tokenize inputs and outputs correctly
def tokenize_data(sample):
    inputs = tokenizer(sample["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(sample["target_text"], padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"]  # Target labels should be tokenized target text
    }

# Apply tokenization to the dataset
dataset = dataset.map(tokenize_data)

# Show a sample
print(dataset[0])


Map:   0%|          | 0/2097 [00:00<?, ? examples/s]

{'input_text': 'fix ASR: professor floyd pointed out that mass lesions are a possible cause for epileptic siigures', 'target_text': ' Professor Floyd pointed out that mass lesions are a possible cause for epileptic seizures', 'input_ids': [2210, 71, 6857, 10, 5812, 8882, 63, 26, 8744, 91, 24, 3294, 110, 2865, 33, 3, 9, 487, 1137, 21, 9241, 40, 14629, 108, 23, 122, 7665, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
from torch.utils.data import DataLoader
import torch

# Set format for PyTorch tensors
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Split into training and validation sets
split = dataset.train_test_split(test_size=0.1)  # 90% training, 10% validation
train_dataset = split["train"]
val_dataset = split["test"]

# Define batch size
batch_size = 8  # Reduce if memory issues occur

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Check sample batch
for batch in train_dataloader:
    print({key: batch[key].shape for key in batch})
    break


{'input_ids': torch.Size([8, 128]), 'attention_mask': torch.Size([8, 128]), 'labels': torch.Size([8, 128])}


In [12]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the T5-small model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5_asr_correction",  # Where to save the model
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",  # Save model after each epoch
    per_device_train_batch_size=8,  # Adjust based on GPU memory
    per_device_eval_batch_size=8,
    learning_rate=3e-4,  # Learning rate
    num_train_epochs=5,  # Number of training epochs
    weight_decay=0.01,  # Regularization
    save_total_limit=2,  # Keep only the last 2 checkpoints
    logging_dir="./logs",  # Log directory
    logging_steps=500,  # Log progress every 500 steps
    push_to_hub=False,  # Set to True if uploading to Hugging Face
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: divyamoduga (divyamoduga-mahindra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.092449
2,No log,0.082731
3,0.225300,0.080877


Epoch,Training Loss,Validation Loss
1,No log,0.092449
2,No log,0.082731
3,0.225300,0.080877
4,0.225300,0.082246
5,0.069500,0.081119


TrainOutput(global_step=1180, training_loss=0.1342874834092997, metrics={'train_runtime': 12089.1885, 'train_samples_per_second': 0.78, 'train_steps_per_second': 0.098, 'total_flos': 319237474222080.0, 'train_loss': 0.1342874834092997, 'epoch': 5.0})

In [13]:
import os

# Create a directory to save the model
save_directory = "./t5_asr_correction_model"
os.makedirs(save_directory, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")


Model saved to ./t5_asr_correction_model


In [43]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load saved model
model_path = "./t5_asr_correction_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Function to predict corrected text
def correct_asr_errors(asr_text):
    input_text = f"fix ASR: {asr_text}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids)
    corrected_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return corrected_text

# Test with examples
asr_sentences = [
    "professor floyd pointed out that mass lesions are a possible cause for epileptic siigures",
    "Turn off the the lights", " sadhika is kool"
]

for sentence in asr_sentences:
    print(f"ASR Output: {sentence}")
    print(f"Corrected: {correct_asr_errors(sentence)}\n")


ASR Output: professor floyd pointed out that mass lesions are a possible cause for epileptic siigures
Corrected: Professor Floyd pointed out that mass lesions are a possible cause for epileptic seizures

ASR Output: Turn off the the lights
Corrected: Turn off the lights

ASR Output:  sadhika is kool
Corrected: Sadhika is a kool



In [16]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.0 MB/s eta 0:00:00


In [18]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import jiwer  # For error rate metrics

# Load trained model and tokenizer
model_path = "/content/t5_asr_correction_model"  # Update with your saved model path
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load test dataset
df = pd.read_excel("eng_data.xlsx")  # Ensure correct file path
asr_outputs = df["asr_output"].tolist()
corrected_texts = df["corrected_text"].tolist()

# Function to generate predictions
def generate_predictions(model, tokenizer, texts):
    predictions = []
    for text in texts:
        input_text = "fix ASR: " + text  # Input prompt for T5 model
        input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)

        # Generate output
        output_ids = model.generate(input_ids, max_length=128)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predictions.append(output_text)

    return predictions

# Get model predictions
predictions = generate_predictions(model, tokenizer, asr_outputs)

# Compute Word Error Rate (WER)
wer = jiwer.wer(corrected_texts, predictions)

# Compute Character Error Rate (CER)
cer = jiwer.cer(corrected_texts, predictions)

# Compute Sentence Error Rate (SER)
ser = sum(1 for ref, hyp in zip(corrected_texts, predictions) if ref != hyp) / len(corrected_texts)

# Print results
print(f"WER: {wer:.2%}")
print(f"CER: {cer:.2%}")
print(f"SER: {ser:.2%}")


WER: 8.87%
CER: 3.90%
SER: 100.00%
